# Creating additional regressors to use with Prophet in other notebooks

## Import necessary modules

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib inline

In [2]:
# function to select a few rows of data, 
# convert to a Pandas dataframe, and transpose

def preview(df, n=3, sample=False, sample_frac=0.01):
    if sample == True:
        return pd.DataFrame(df.sample(False, sample_frac).take(n), columns=df.columns)
    else:
        return pd.DataFrame(df.take(n), columns=df.columns)

## Open files

### 2014 files

In [5]:
# create list of names of files to open

fp = 'gs://metis_bucket_av/data/citibike/with_loc/'
months = ['201501', '201502', '201503', '201504', '201505', '201506']
fn = '{}-citibike-tripdata.csv'
fns = [fp + fn.format(m) for m in months]

In [23]:
# open each CSV, read in with spark, 
# and join to create one large dataframe

df = spark.read.csv(fns[0], header=True)
for f in fns[1:]:
    temp_df = spark.read.csv(f, header=True)
    df = df.union(temp_df)

In [7]:
# check # of rows in resulting final dataframe

num_rows_df = df.count()
print('{:,} rows'.format(num_rows_df))

3,379,903 rows


In [9]:
preview(df, 5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_loc
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.75001986,-73.96905301,265,Stanton St & Chrystie St,40.72229346,-73.99147535,18660,Subscriber,1960.0,2,Midtown
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.74317449,-74.00366443,482,W 15 St & 7 Ave,40.73935542,-73.99931783,16085,Subscriber,1963.0,1,Chelsea
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.74096374,-73.98602213,505,6 Ave & W 33 St,40.74901271,-73.98848395,20845,Subscriber,1974.0,1,Flatiron District
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.68317813,-73.9659641,399,Lafayette Ave & St James Pl,40.68851534,-73.9647628,19610,Subscriber,1969.0,1,Clinton Hill
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.7451677,-73.98683077,432,E 7 St & Avenue A,40.72621788,-73.98379855,20197,Subscriber,1977.0,1,Midtown


In [11]:
df.printSchema()

root
 |-- tripduration: string (nullable = true)
 |-- starttime: string (nullable = true)
 |-- stoptime: string (nullable = true)
 |-- start station id: string (nullable = true)
 |-- start station name: string (nullable = true)
 |-- start station latitude: string (nullable = true)
 |-- start station longitude: string (nullable = true)
 |-- end station id: string (nullable = true)
 |-- end station name: string (nullable = true)
 |-- end station latitude: string (nullable = true)
 |-- end station longitude: string (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth year: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- start_loc: string (nullable = true)



## Casting data types

In [12]:
import pyspark.sql.types as pst
import pyspark.sql.functions as psf

In [25]:
df = df.select(
        psf.to_timestamp(df['starttime'], 'M/d/yyyy H:mm').alias('dt'),
        df.start_loc)

df.printSchema()

root
 |-- dt: timestamp (nullable = true)
 |-- start_loc: string (nullable = true)



## Converting to Pandas

In [34]:
cb = (df
      .groupby(psf.date_format("dt", 'yyyy-MM-dd HH').alias("date"))
      .pivot('start_loc').count()
      .orderBy('date').toPandas())

In [38]:
cb.set_index('date', inplace=True)

In [40]:
cb.head()

,Battery Park City,Bedford-Stuyvesant,Boerum Hill,Brooklyn Heights,Central Park,Chelsea,Chinatown,Civic Center,Clinton Hill,Columbia St,...,Stuyvesant Town,Sunset Park,Theater District,Tribeca,Two Bridges,Upper East Side,Upper West Side,Vinegar Hill,West Village,Williamsburg
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00,2.0,NaN,2.0,2.0,NaN,17.0,2.0,1.0,6.0,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,3.0,NaN,4.0,2.0
2015-01-01 01,NaN,3.0,2.0,2.0,NaN,20.0,5.0,2.0,3.0,NaN,...,5.0,NaN,NaN,5.0,NaN,NaN,2.0,NaN,10.0,2.0
2015-01-01 02,1.0,NaN,3.0,1.0,NaN,17.0,2.0,NaN,11.0,NaN,...,3.0,NaN,NaN,4.0,NaN,NaN,1.0,NaN,11.0,3.0
2015-01-01 03,NaN,2.0,1.0,1.0,NaN,14.0,1.0,1.0,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,8.0,1.0
2015-01-01 04,NaN,1.0,NaN,NaN,NaN,4.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,5.0,NaN


## Write to a local CSV file

In [47]:
cb.to_csv('citibike_hourly.csv')